### **CRIADO O BANCO DE DADOS DW\_06**

In [1]:
--CRIAR O DB LOCAL (comando executado no MSSMS)
--CREATE DATABASE DW_06
--GO


--SELECIONANDO O BANCO DE DADOS
USE [DW_06]
GO

Commands completed successfully.

Total execution time: 00:00:00

### **CRIANDO UMA TABELA DE LOG**

In [2]:
USE [DW_06]
GO


IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='LOG_CARGAS' AND xtype='U')

CREATE TABLE LOG_CARGAS
 (ID INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
 NUMERO_ERRO INT,
 SERVERIDADE_ERRO INT,
 ESTADO_ERRO  INT,
 PROC_ERRO NVARCHAR(255),
 LINHA_ERRO INT,
 MSG_ERRO NVARCHAR(255),
 SITUACAO NVARCHAR(10),
 PROCESSO NVARCHAR(30),
 DATA_INI DATETIME,
 DATA_FIM DATETIME  
 )

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

### **CRIANDO A TABELA STG**

In [27]:
--SELECIONANDO O BANCO DE DADOS
USE [DW_06]
GO


IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='STG_VENDAS' AND xtype='U')
CREATE TABLE [dbo].[STG_VENDAS](
	[Segmento] [varchar](50) NULL,
	[Pais] [varchar](50) NULL,
    [Produto] [varchar](50) NULL,
	[TipoDesconto] [varchar](50) NULL,
	[QtdeVendas] [varchar](50) NULL,
	[PrecoCusto] [varchar](50) NULL,
	[PrecoVenda] [varchar](50) NULL,
	[VendasBruta] [varchar](50) NULL,
	[Desconto] [varchar](50) NULL,
	[Vendas] [varchar](50) NULL,
	[CustoVenda] [varchar](50) NULL,
	[Lucro] [varchar](50) NULL,
	[DataVenda] [varchar](50) NULL,
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00

**TRUNCANDO OS DADOS DA TABELA**

In [5]:
TRUNCATE TABLE STG_VENDAS;

Commands completed successfully.

Total execution time: 00:00:00.005

**CARREGANDO DADOS NA TABELA STAGIN COM BULK INSERT**

In [28]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  

BULK INSERT STG_VENDAS
FROM 'C:\Users\Marcia\OneDrive\Área de Trabalho\DataEX\BaseDadosDesafio.csv'
WITH        (
	codepage='65001',  -- { 'ACP' | 'OEM' | 'RAW' | 'code_page' } ] 65001
    DATAFILETYPE = 'char',   --      { 'char' | 'native'| 'widechar' | 'widenative' } ]         
	fieldterminator=';',               
	rowterminator='\n',               
	maxerrors = 0,     --qtde erros aceitaveis          
	fire_triggers,             
    firstrow = 2           --linha onde começam os dados, desconsiderando o cabeçalho    
   --lastrow = 10 --atributo de quantas linhas pegar
	     ) 
END TRY  
BEGIN CATCH  --se der erro ele executa esses comandos
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS --se der tudo certo roda essa parte
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(700 rows affected)

(1 row affected)

Total execution time: 00:00:00.039

### **CRIANDO AS TABELAS DIMENSÃO**

In [10]:
USE [DW_06]
GO


--CRIANDO DIMENSAO SEGMENTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Segmento' AND xtype='U')
CREATE TABLE [dbo].[D_Segmento](
	[Id_Segmento] [int] IDENTITY(1,1) NOT NULL,
	[Nome_Segmento] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_Segmento] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO


--CRIANDO DIMENSAO PAÍS
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Pais' AND xtype='U')
CREATE TABLE [dbo].[D_Pais](
	[Id_Pais] [int] IDENTITY(1,1) NOT NULL,
	[Nome_Pais] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_Pais] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO


--CRIANDO DIMENSAO PRODUTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Produto' AND xtype='U')
CREATE TABLE [dbo].[D_Produto](
	[Id_Produto] [int] IDENTITY(1,1) NOT NULL,
	[Nome_Produto] [varchar](50) NULL,
	[Preco_Custo] [decimal](10, 2) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_Produto] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO


--CRIANDO DIMENSAO TIPO DESCONTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Tipo_Desconto' AND xtype='U')
CREATE TABLE [dbo].[D_Tipo_Desconto](
	[Id_Tipo_Desconto] [int] IDENTITY(1,1) NOT NULL,
	[Nome_Tipo_Desconto] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_Tipo_Desconto] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.038

**CARGA DIMENSÃO SEGMENTO**

In [4]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA SEGMENTO'
BEGIN TRY  

MERGE D_Segmento AS Destino

USING (SELECT DISTINCT Segmento FROM STG_VENDAS) AS Origem

ON Destino.Nome_Segmento = Origem.Segmento

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET Nome_Segmento = Origem.Segmento

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (Nome_Segmento) VALUES (Origem.Segmento);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.023

**CARGA DIMENSÃO PAÍS**

In [19]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA PAIS'
BEGIN TRY  

MERGE D_Pais AS Destino

USING (SELECT DISTINCT Pais FROM STG_VENDAS) AS Origem

ON Destino.Nome_Pais = Origem.Pais

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET Nome_Pais = Origem.Pais

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (Nome_Pais) VALUES (Origem.Pais);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.029

**CARGA DIMENSÃO PRODUTO**

In [22]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA PRODUTO'
BEGIN TRY  

MERGE D_Produto AS Destino

USING (SELECT DISTINCT Produto, PrecoCusto FROM STG_VENDAS) AS Origem

ON Destino.Nome_Produto = Origem.Produto
AND Destino.Preco_Custo = Origem.PrecoCusto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET Nome_Produto = Origem.Produto,
    Preco_Custo = Origem.PrecoCusto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (Nome_Produto, Preco_Custo) VALUES (Origem.Produto, Origem.PrecoCusto);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO 

(6 rows affected)

(1 row affected)

Total execution time: 00:00:00.016

**CARGA DIMENSÃO TIPO DESCONTO**

In [24]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA TIPO DESCONTO'
BEGIN TRY  

MERGE D_Tipo_Desconto AS Destino

USING (SELECT DISTINCT TipoDesconto FROM STG_VENDAS) AS Origem

ON Destino.Nome_Tipo_Desconto = Origem.TipoDesconto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET Nome_Tipo_Desconto = Origem.TipoDesconto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (Nome_Tipo_Desconto) VALUES (Origem.TipoDesconto);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(4 rows affected)

(1 row affected)

Total execution time: 00:00:00.023

**LIMPA A TABELA FATO**

In [57]:
TRUNCATE TABLE FATO_VENDAS;

Commands completed successfully.

Total execution time: 00:00:00.006

**CRIANDO A TABELA FATO**

In [58]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
	SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA FATO_VENDAS'
BEGIN TRY 

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='FATO_VENDAS' AND xtype='U')
CREATE TABLE FATO_VENDAS
(
Id_Segmento  [int] NULL,
Id_Pais  [int] NULL,
Id_Produto  [int] NULL,
Id_Tipo_Desconto  [int] NULL,
Qtde_Vendas [int] NULL,
Preco_Venda DECIMAL(10,2) NULL,
Vendas_Bruta DECIMAL(10,2) NULL,
Desconto DECIMAL(10,2) NULL,
Vendas DECIMAL(10,2) NULL,
Custo_Venda DECIMAL(10,2) NULL,
Lucro DECIMAL(10,2) NULL,
Data_Venda  DATE
)

END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
           [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(1 row affected)

Total execution time: 00:00:00.012

**ANALISANDO AMOSTRAGEM DOS DADOS**

In [59]:
SELECT  seg.Id_Segmento,
        pa.Id_Pais,
        prd.Id_Produto,
        prd.Preco_Custo,
        td.Id_Tipo_Desconto,
        stg.QtdeVendas,
        stg.PrecoCusto,
        stg.PrecoVenda,
        stg.VendasBruta,
        stg.Desconto,
        stg.Vendas,
        stg.CustoVenda,
        stg.Lucro,
        stg.DataVenda

FROM STG_VENDAS stg
 INNER JOIN D_Segmento seg
ON seg.Nome_Segmento = stg.Segmento

 INNER JOIN D_Pais pa
ON pa.Nome_Pais = stg.Pais

 INNER JOIN D_Produto prd
ON prd.Nome_Produto = stg.Produto

 INNER JOIN D_Tipo_Desconto td
ON td.Nome_Tipo_Desconto = stg.TipoDesconto



(700 rows affected)

Total execution time: 00:00:00.053

Id_Segmento,Id_Pais,Id_Produto,Preco_Custo,Id_Tipo_Desconto,QtdeVendas,PrecoCusto,PrecoVenda,VendasBruta,Desconto,Vendas,CustoVenda,Lucro,DataVenda
3,3,2,"3,00",4,1513,3,350,529550,0,529550,393380,136170,01/12/2014
3,3,4,"10,00",4,1006,10,350,352100,0,352100,261560,90540,01/06/2014
3,1,4,"10,00",4,1725,10,350,603750,0,603750,448500,155250,01/11/2013
3,3,4,"10,00",4,1513,10,350,529550,0,529550,393380,136170,01/12/2014
3,3,5,"120,00",4,1006,120,350,352100,0,352100,261560,90540,01/06/2014
3,2,6,"250,00",4,1527,250,350,534450,0,534450,397020,137430,01/09/2013
3,2,1,"260,00",4,2750,260,350,962500,0,962500,715000,247500,01/02/2014
3,4,2,"3,00",2,1210,3,350,423500,4235,419265,314600,104665,01/03/2014
3,4,2,"3,00",2,1397,3,350,488950,"4889,5","484060,5",363220,"120840,5",01/10/2014
3,2,2,"3,00",2,2155,3,350,754250,"7542,5","746707,5",560300,"186407,5",01/12/2014
